In [1]:
import open3d as o3d
import numpy as np

from skspatial.objects import Plane, Points
from skspatial.plotting import plot_3d
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime

import os
import pandas as pd


INFO - 2022-12-01 18:38:04,248 - utils - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 2022-12-01 18:38:04,249 - utils - NumExpr defaulting to 8 threads.


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# pcd = o3d.io.read_point_cloud('../../maps/flat.ply')
# assert isinstance(pcd, o3d.geometry.PointCloud)
# print(pcd)

In [3]:
# pcds: list[o3d.geometry.PointCloud] = []
plys_dir = '../../maps/flat_frame_separated/'

pc_fns = [fn for fn in os.listdir(plys_dir)]

def get_pc_from_fn(fn_:str)-> o3d.geometry.PointCloud:
    return o3d.io.read_point_cloud(plys_dir+fn_)

In [99]:
DIST_THRESHOLD= 0.1

def get_outAndin_clouds(pc_):
    _pc = pc_.voxel_down_sample(voxel_size=0.1)

    _pcd, _ = _pc.remove_statistical_outlier(nb_neighbors=10, std_ratio=1.0)

    plane_model, inliers = _pcd.segment_plane(distance_threshold=DIST_THRESHOLD,
                                                ransac_n=3,
                                                num_iterations=1000)

    inlier_cloud = _pcd.select_by_index(inliers)
    outlier_cloud = _pcd.select_by_index(inliers, invert=True)

    return outlier_cloud, inlier_cloud

def plot_outliers_count(outliers_pc_:list, inliers_pc_:list=[],idx_range_:tuple=()):
    # slicing
    outliers_pc_ = outliers_pc_ [idx_range_[0]:idx_range_[1]] if idx_range_ else outliers_pc_  
    inliers_pc_ = inliers_pc_ [idx_range_[0]:idx_range_[1]] if (idx_range_ and inliers_pc_)  else inliers_pc_ 

    _out_np = np.array([outliers_pc_]).transpose()
    _out_np = np.concatenate((_out_np,np.zeros((_out_np.shape[0],1))),axis=1)
    _out_df = pd.DataFrame(_out_np, columns=['point_count','category'])
    _out_df[['category']] = 'outliers'

    if inliers_pc_:
        _inliers_np = np.array([inliers_pc_]).transpose()
        _inliers_np = np.concatenate((_inliers_np,np.zeros((_inliers_np.shape[0],1))),axis=1)

        _indf = pd.DataFrame(_inliers_np, columns=['point_count','category'])
        _indf[['category']] = 'inliers'
        df = pd.concat((_indf, _out_df))
    else:
        df = _out_df

    fig = px.bar(df, x=df.index, y="point_count", color="category", barmode="group", text_auto=True)
    # fig = px.histogram(df, x="point_count", color="category")
    fig.show()
    return df

def plot_pc(pc_, display_o3d_=False):
    out_cloud, in_cloud = get_outAndin_clouds(pc_)

    if display_o3d_:
        in_cloud.paint_uniform_color([0.8, 0.5, 0])
        out_cloud.paint_uniform_color([1, 0, 0])
        o3d.visualization.draw_geometries([in_cloud, out_cloud])

    inliers_array = np.array([p for i,p in enumerate(np.asarray(in_cloud.points))])
    outliers10_array = np.array([p for i,p in enumerate(np.asarray(out_cloud.points))])
    
    print(f"This frame has {outliers10_array.shape[0]} points more than {DIST_THRESHOLD} m away from plane")

    inliers_array = np.concatenate((inliers_array, np.zeros((inliers_array.shape[0],1))),axis=1)
    outliers10_array = np.concatenate((outliers10_array, np.ones((outliers10_array.shape[0],1))),axis=1)

    indf = pd.DataFrame(inliers_array, columns=['X','Y','Z','category'])
    out10df = pd.DataFrame(outliers10_array, columns=['X','Y','Z','category'])

    indf[['category']] = 'inliers'
    out10df[['category']] = 'outliers'
    df = pd.concat((indf, out10df))


    fig = px.scatter_3d(df,x='X',y='Y',z='Z',color='category')
    fig.update_traces(marker=dict(size=3))
    fig.show()
    # fig.write_image('test.png')
    # fig.write_html('test.html')

def plot_inNout_pc(out_cloud, in_cloud, display_o3d_=False):
    
    if display_o3d_:
        in_cloud.paint_uniform_color([0.8, 0.5, 0])
        out_cloud.paint_uniform_color([1, 0, 0])
        o3d.visualization.draw_geometries([in_cloud, out_cloud])
        return

    inliers_array = np.array([p for i,p in enumerate(np.asarray(in_cloud.points))])
    outliers10_array = np.array([p for i,p in enumerate(np.asarray(out_cloud.points))])
    
    print(f"This frame has {outliers10_array.shape[0]} points more than {DIST_THRESHOLD} m away from plane")

    inliers_array = np.concatenate((inliers_array, np.zeros((inliers_array.shape[0],1))),axis=1)
    outliers10_array = np.concatenate((outliers10_array, np.ones((outliers10_array.shape[0],1))),axis=1)

    indf = pd.DataFrame(inliers_array, columns=['X','Y','Z','category'])
    out10df = pd.DataFrame(outliers10_array, columns=['X','Y','Z','category'])

    indf[['category']] = 'inliers'
    out10df[['category']] = 'outliers'
    df = pd.concat((indf, out10df))


    fig = px.scatter_3d(df,x='X',y='Y',z='Z',color='category')
    fig.update_traces(marker=dict(size=3))
    fig.show()
    # fig.write_image('test.png')
    # fig.write_html('test.html')


In [ ]:
# analyze individual frames
inliers_pointcount = []
out_pointcount = []
for i, fn in enumerate(pc_fns):
    _pcd = get_pc_from_fn(fn)

    # Get out- and in-liers
    out_cloud_n, in_cloud_n = get_outAndin_clouds(_pcd)

    # Coloring
    in_cloud_n.paint_uniform_color([0, 0, 0])
    out_cloud_n.paint_uniform_color([0.8, 0.5, 0])

    # analysis
    # count how many points in outlier cloud
    inliers_pointcount.append(len(in_cloud_n.points))
    out_pointcount.append(len(out_cloud_n.points))

    if i % 50 == 0:
        print(f'Finished {i+1}/{len(pc_fns)}')

In [34]:
df = plot_outliers_count(out_pointcount, inliers_pointcount)

In [35]:
# # How many frames with more than C outliers in frame? 
# filter point-count lists
C = 40
_filtered_outliers_count =[]
_filtered_inliers_count = []
for (_out_count, _in_count) in zip(out_pointcount, inliers_pointcount):
    if _out_count>C:
        _filtered_outliers_count.append(_out_count)
        _filtered_inliers_count.append(_in_count)

# df = plot_outliers_count(_filtered_outliers_count, _filtered_inliers_count)

print(f"There are '{len(_filtered_outliers_count)}' frames with more than '{C}' points '{DIST_THRESHOLD}' m away from plane")

filtered_df = df.loc[(df['category'] == 'outliers') & (df['point_count'] > C)]
print(f"Filtered indices: {filtered_df.index}")
filtered_idxs_gen = (idx for idx in filtered_df.index)

There are '79' frames with more than '40' points '0.1' m away from plane
Filtered indices: Int64Index([  0,   1,   7,   8,  13,  14,  16,  18,  21,  26,  27,  28,  29,
             33,  36,  44,  45,  46,  48,  49,  53,  66,  70,  72,  73,  75,
             77,  81,  83,  84,  85,  86,  95,  96, 100, 102, 105, 106, 108,
            116, 126, 128, 132, 140, 145, 146, 150, 152, 153, 155, 156, 157,
            159, 160, 163, 168, 170, 175, 181, 185, 196, 200, 201, 216, 217,
            218, 222, 224, 227, 229, 230, 235, 237, 238, 240, 241, 246, 249,
            251],
           dtype='int64')


In [103]:
def cloud_stats(pc_):
    
    cloud_array = np.array([p for p in np.asarray(pc_.points)])

    # Statistical analysis
    return cloud_array.shape, np.cov(cloud_array.transpose())
    
# _i = next(filtered_idxs_gen)

def full_pc_analysis(c_n_, plot_=True):
    print(f"getting pc #{c_n_}")
    _pc_n = get_pc_from_fn(pc_fns[c_n_])
    _out_cloud, _in_cloud = get_outAndin_clouds(_pc_n)

    shape_out_n, cov_out_n = cloud_stats(_out_cloud)
    shape_in_n, cov_in_n = cloud_stats(_in_cloud)

    print(f"Cloud {c_n_} has the following stats:")
    print(f"Shape: Outliers:{shape_out_n}, Inliers:{shape_in_n}")
    # print('Covarianve:')
    # print(f"Outliers:\n{cov_out_n}")
    # print(f"Inliers:\n{cov_in_n}")
    
    if plot_:
        plot_inNout_pc(_out_cloud, _in_cloud, False)
    
    return _pc_n, _out_cloud, _in_cloud


# _i = next(filtered_idxs_gen)
_i = 77
pc_, out_cloud, in_cloud = full_pc_analysis(_i)

getting pc #77
Cloud 77 has the following stats:
Shape: Outliers:(105, 3), Inliers:(2454, 3)
This frame has 105 points more than 0.1 m away from plane


In [98]:
plot_inNout_pc(out_cloud, in_cloud, True)